<a href="https://colab.research.google.com/github/bumbum2403/Zeotap_Assessment/blob/main/Shubham_Dwivedi_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

In [9]:
# Merge Transactions with Products to include product details
transactions_products = pd.merge(transactions, products, on="ProductID", how="left")

# Merge the result with Customers to include customer details
full_data = pd.merge(transactions_products, customers, on="CustomerID", how="left")

print("Merged Dataset:")
print(full_data.head())

# Save the merged dataset to inspect if needed
full_data.to_csv("Merged_Dataset.csv", index=False)


Merged Dataset:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x                      ProductName     Category  Price_y  \
0      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
1      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
2      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
3      601.36   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
4      902.04   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   

      CustomerName         Region  SignupDate  
0   Andrea Jenkins    

In [7]:
full_data.columns


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'ProductName', 'Category',
       'Price_y', 'CustomerName', 'Region', 'SignupDate'],
      dtype='object')

The 2 columns, price_x and price_y have exactly same values in them, so let's drop one of them, and rename the other as Price only for generating better data

In [11]:
full_data = full_data.drop(columns=["Price_x"]).rename(columns={"Price_y": "Price"})

print("Updated Merged Dataset (Duplicate Price Column Removed):")
print(full_data.head())


Updated Merged Dataset (Duplicate Price Column Removed):
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue                      ProductName     Category   Price  \
0      300.68  ComfortLiving Bluetooth Speaker  Electronics  300.68   
1      300.68  ComfortLiving Bluetooth Speaker  Electronics  300.68   
2      300.68  ComfortLiving Bluetooth Speaker  Electronics  300.68   
3      601.36  ComfortLiving Bluetooth Speaker  Electronics  300.68   
4      902.04  ComfortLiving Bluetooth Speaker  Electronics  300.68   

      CustomerName         Region  SignupDate  
0   Andrea Jenkins         Europe  2022-1

# FEATURE ENGINEERING:
1.   Total Transaction Value: Sum of all transaction values for the customer.
2. Average Transaction Value: Average spending per transaction.
3. Transaction Frequency: Total number of transactions.
4. Preferred Product Category: The product category the customer spends the most on.
5. Average Price per Product: Mean price of products purchased by the customer.

In [12]:
# Total Transaction Value per customer
total_transaction_value = full_data.groupby("CustomerID")["TotalValue"].sum().reset_index()
total_transaction_value.rename(columns={"TotalValue": "TotalTransactionValue"}, inplace=True)

# Average Transaction Value per customer
average_transaction_value = full_data.groupby("CustomerID")["TotalValue"].mean().reset_index()
average_transaction_value.rename(columns={"TotalValue": "AverageTransactionValue"}, inplace=True)

# Transaction Frequency per customer
transaction_frequency = full_data.groupby("CustomerID")["TransactionID"].count().reset_index()
transaction_frequency.rename(columns={"TransactionID": "TransactionFrequency"}, inplace=True)

# Preferred Product Category per customer
preferred_category = (
    full_data.groupby(["CustomerID", "Category"])["TotalValue"]
    .sum()
    .reset_index()
    .sort_values(["CustomerID", "TotalValue"], ascending=[True, False])
)
preferred_category = preferred_category.drop_duplicates("CustomerID")[["CustomerID", "Category"]]
preferred_category.rename(columns={"Category": "PreferredCategory"}, inplace=True)

# Average Price per Product per customer
average_price_per_product = full_data.groupby("CustomerID")["Price"].mean().reset_index()
average_price_per_product.rename(columns={"Price": "AveragePricePerProduct"}, inplace=True)

# Combine all features into a single DataFrame
customer_features = total_transaction_value.merge(average_transaction_value, on="CustomerID")
customer_features = customer_features.merge(transaction_frequency, on="CustomerID")
customer_features = customer_features.merge(preferred_category, on="CustomerID")
customer_features = customer_features.merge(average_price_per_product, on="CustomerID")

# Display the features
print("Customer-Level Features:")
print(customer_features.head())

customer_features.to_csv("Customer_Features.csv", index=False)


Customer-Level Features:
  CustomerID  TotalTransactionValue  AverageTransactionValue  \
0      C0001                3354.52                  670.904   
1      C0002                1862.74                  465.685   
2      C0003                2725.38                  681.345   
3      C0004                5354.88                  669.360   
4      C0005                2034.24                  678.080   

   TransactionFrequency PreferredCategory  AveragePricePerProduct  
0                     5       Electronics              278.334000  
1                     4          Clothing              208.920000  
2                     4       Electronics              195.707500  
3                     8        Home Decor              240.636250  
4                     3       Electronics              291.603333  


Let's Use Cosine Similarity to measure similarity between customer profiles.
Extracting feature columns only (numeric values) for the calculation.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import pandas as pd

# let's  Normalize Features first for a better fit
# and Select numeric feature columns
feature_cols = ["TotalTransactionValue", "AverageTransactionValue", "TransactionFrequency", "AveragePricePerProduct"]

# Normalize numeric features using StandardScaler
scaler = StandardScaler()
normalized_profiles = scaler.fit_transform(customer_features[feature_cols])

# Computing cosine similarity between normalized customer profiles
similarity_matrix = cosine_similarity(normalized_profiles)

# Convert similarity matrix to a DataFrame
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=customer_features["CustomerID"],
    columns=customer_features["CustomerID"]
)

# Step 3: Generate Lookalikes for First 20 Customers
lookalikes = {}
customer_subset = customer_features[customer_features["CustomerID"].isin([f"C{str(i).zfill(4)}" for i in range(1, 21)])]

for cust_id in customer_subset["CustomerID"]:
    # Get similarity scores for the current customer, excluding itself
    similar_customers = similarity_df[cust_id].sort_values(ascending=False).iloc[1:4]
    # Store top 3 similar customers and their scores
    lookalikes[cust_id] = [{"cust_id": idx, "score": round(score, 4)} for idx, score in similar_customers.items()]

# Converting lookalikes dictionary to a DataFrame and downloading it
lookalike_df = pd.DataFrame({
    "cust_id": list(lookalikes.keys()),
    "lookalikes": [str(v) for v in lookalikes.values()]
})

lookalike_df.to_csv("Lookalike.csv", index=False)

print("Lookalike Recommendations:")
print(lookalike_df.head())


Lookalike Recommendations:
  cust_id                                         lookalikes
0   C0001  [{'cust_id': 'C0103', 'score': 0.9744}, {'cust...
1   C0002  [{'cust_id': 'C0029', 'score': 0.9996}, {'cust...
2   C0003  [{'cust_id': 'C0178', 'score': 0.9539}, {'cust...
3   C0004  [{'cust_id': 'C0075', 'score': 0.9971}, {'cust...
4   C0005  [{'cust_id': 'C0061', 'score': 0.9922}, {'cust...
